In [1]:
import pandas as pd
import math
import altair as alt

# Stop, question and frisk data

Source: NYPD

## Load data

In [2]:
df_sqf = pd.read_excel("data/raw/sqf-2019.xlsx", usecols=[0, 29, 30, 78, 79, 82])
df_sqf.columns = df_sqf.columns.str.lower()

df_sqf.info()
df_sqf.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13459 entries, 0 to 13458
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   stop_id_anony            13459 non-null  int64 
 1   frisked_flag             13459 non-null  object
 2   searched_flag            13459 non-null  object
 3   stop_location_x          13459 non-null  int64 
 4   stop_location_y          13459 non-null  int64 
 5   stop_location_boro_name  13459 non-null  object
dtypes: int64(3), object(3)
memory usage: 631.0+ KB


,stop_id_anony,frisked_flag,searched_flag,stop_location_x,stop_location_y,stop_location_boro_name
0,1,Y,N,979667,199737,MANHATTAN
1,2,N,Y,982650,201326,MANHATTAN
2,3,N,N,984063,203033,MANHATTAN
3,4,N,N,982848,202677,MANHATTAN
4,5,Y,N,983100,202705,MANHATTAN


## Calculate top-line statistics (all boroughs)

In [3]:
#  From https://realpython.com/python-rounding/#rounding-half-up


def round_half_up(n, decimals=0):
    multiplier = 10 ** decimals

    return math.floor(n * multiplier + 0.5) / multiplier

In [4]:
stopped_count = df_sqf["stop_id_anony"].nunique()
frisked_count = df_sqf.query("frisked_flag == 'Y'")["stop_id_anony"].nunique()
searched_count = df_sqf.query("searched_flag == 'Y'")["stop_id_anony"].nunique()
frisked_searched_count = df_sqf.query("frisked_flag == 'Y' or searched_flag == 'Y'")[
    "stop_id_anony"
].nunique()
frisked_pc = round_half_up(frisked_count / stopped_count * 100, 1)
searched_pc = round_half_up(searched_count / stopped_count * 100, 1)
frisked_searched_pc = round_half_up(frisked_searched_count / stopped_count * 100, 1)

print(
    f"""
Number stopped: {stopped_count}
Number frisked: {frisked_count}
Number searched: {searched_count}
Number frisked or searched: {frisked_searched_count}
Per cent frisked: {frisked_pc}
Per cent searched: {searched_pc}
Per cent frisked or searched: {frisked_searched_pc}"""
)


Number stopped: 13459
Number frisked: 7624
Number searched: 4988
Number frisked or searched: 9227
Per cent frisked: 56.6
Per cent searched: 37.1
Per cent frisked or searched: 68.6


## Export to `.csv` for counting points in polygons in QGIS

In [5]:
df_sqf.to_csv("data/processed/stop-question-frisk-2019.csv", index=False)

## Load stops count exported from QGIS (three boroughs; includes parks)

In [6]:
df_stops = (
    pd.read_csv(
        "data/processed/num-stops-by-nta.csv",
        usecols=[0, 1, 3, 4, 7],
        header=0,
        names=["borough_code", "borough_name", "nta_code", "nta_name", "num_stops"],
    )
    .query("borough_name in ['Bronx', 'Brooklyn', 'Manhattan']")
    .sort_values("nta_code")
    .reset_index(drop=True)
)

df_stops.info()
df_stops.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118 entries, 0 to 117
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   borough_code  118 non-null    int64 
 1   borough_name  118 non-null    object
 2   nta_code      118 non-null    object
 3   nta_name      118 non-null    object
 4   num_stops     118 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 4.7+ KB


,borough_code,borough_name,nta_code,nta_name,num_stops
0,3,Brooklyn,BK09,Brooklyn Heights-Cobble Hill,29
1,3,Brooklyn,BK17,Sheepshead Bay-Gerritsen Beach-Manhattan Beach,65
2,3,Brooklyn,BK19,Brighton Beach,44
3,3,Brooklyn,BK21,Seagate-Coney Island,153
4,3,Brooklyn,BK23,West Brighton,15


In [7]:
df_stops["num_stops"].sum()

10498

## Calculate stops by borough

In [8]:
bronx_stopped = df_stops.query("borough_name == 'Bronx'")["num_stops"].sum()
brooklyn_stopped = df_stops.query("borough_name == 'Brooklyn'")["num_stops"].sum()
manhattan_stopped = df_stops.query("borough_name == 'Manhattan'")["num_stops"].sum()

print(bronx_stopped, brooklyn_stopped, manhattan_stopped)

2450 4311 3737


## Join to adjusted intersections data

In [9]:
df_intersections_adj = pd.read_csv(
    "data/processed/camera-medians-2021-05-19-ntas-adjusted-recoded.csv"
)

df_intersections_adj.info()
df_intersections_adj.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19396 entries, 0 to 19395
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   intersection_id   19396 non-null  object
 1   borough           19396 non-null  object
 2   public_median     19396 non-null  int64 
 3   private_median    19396 non-null  int64 
 4   total_median      19396 non-null  int64 
 5   adjusted_NTACode  19396 non-null  object
 6   adjusted_NTAName  19396 non-null  object
dtypes: int64(3), object(4)
memory usage: 1.0+ MB


,intersection_id,borough,public_median,private_median,total_median,adjusted_NTACode,adjusted_NTAName
0,AXkw4-uH2iuwklCl-qox,Brooklyn,0,1,1,BK88,Borough Park
1,AXkw40-B2iuwklCl-qao,Brooklyn,0,0,0,BK88,Borough Park
2,AXkw40-x2iuwklCl-qap,Brooklyn,0,1,1,BK88,Borough Park
3,AXkw405zLVwDtjXR1gOB,Brooklyn,0,1,1,BK88,Borough Park
4,AXkw406iLVwDtjXR1gOC,Brooklyn,0,0,0,BK88,Borough Park


## Calculate top-line statistics from joined data

### Overall correlation between stops and intersections/cameras

In [ ]:
# TKTK

### Correlation between stops and intersections/cameras by borough

In [ ]:
# TKTK

### Stops per 100 cameras

In [ ]:
# TKTK